<a href="https://colab.research.google.com/github/TaraRasti/Deep_Learning_F20_Assignments/blob/master/Assignment4/nnModule.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
import numpy as np
import matplotlib.pyplot as plt
from torch.utils import data
from torchvision import datasets
from torch.utils.data import Dataset, DataLoader
from torch.autograd import Variable
import torchvision.transforms as transforms
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.tensorboard import SummaryWriter

pathdata='/content/drive/My Drive/dataFMNIST'

transformations = transforms.Compose([transforms.Resize((32,32)),
                                      transforms.ToTensor(),
                                      transforms.Normalize([.5],[.5])
                                      ])


data=datasets.FashionMNIST(pathdata,train=True, download=True,transform=transformations)

train_set, test_set = torch.utils.data.random_split(data, [42000, 18000])

train_loader = torch.utils.data.DataLoader(train_set,shuffle=True)
test_loader = torch.utils.data.DataLoader(test_set,shuffle=True)

In [2]:
class Network(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(1, 6, 5)
        self.conv2 = nn.Conv2d(6, 12, 5)

        self.fc1 = nn.Linear(in_features=300, out_features=120)
        self.fc2 = nn.Linear(in_features=120, out_features=60)
        self.fc3 = nn.Linear(in_features=60, out_features=30)
        self.out = nn.Linear(in_features=30, out_features=10)

    def forward(self, t):

        t = F.relu(self.conv1(t))
        t = F.max_pool2d(t, kernel_size=2, stride=2)

        t = torch.sigmoid(self.conv2(t))
        t = F.max_pool2d(t, kernel_size=2, stride=2)

        t = t.flatten(start_dim=1)
        t = F.relu(self.fc1(t))
        t = F.relu(self.fc2(t))
        t = F.softmax(self.fc3(t))
        t = self.out(t)

        return t

    def training_step(self, batch):
      images, labels = batch
      out = self(images)
      loss = loss_fn(out, labels)
      return loss

    def validation_step(self, batch):
      images, labels = batch
      out = self(images)
      loss = loss_fn(out, labels)
      acc = accuracy(out, labels)
      return {'test_loss': loss.detach(), 'test_acc':acc.detach()}
  
    def validation_epoch_end(self, outputs):
      batch_losses = [x['test_loss'] for x in outputs]
      epoch_loss = torch.stack(batch_losses).mean()
      batch_accs = [x['test_acc'] for x in outputs]
      epoch_acc = torch.stack(batch_accs).mean()
      return {'test_loss':epoch_loss.item(), 'test_acc':epoch_acc.item()}

    def epoch_end(self, epoch, result):
      print("Epoch [{}], test_loss: {:.4f}, test_acc: {:.4f}".format(epoch, result['test_loss'], result['test_acc']))

In [3]:
model=Network()

In [4]:
def accuracy(outputs, labels):
  _, preds = torch.max(outputs, dim=1)
  return torch.tensor(torch.sum(preds == labels).item()/len(preds))

In [5]:
def evaluate(model, test_loader):
  outputs = [model.validation_step(batch) for batch in test_loader]
  return model.validation_epoch_end(outputs)

def fit(epochs, lr, model, train_loader, test_loader, opt_func=torch.optim.SGD):
  history = []
  optimizer = opt_func(model.parameters(), lr)
  for epoch in range(epochs):
    for batch in train_loader:
      loss = model.training_step(batch)
      loss.backward()
      optimizer.step()
      optimizer.zero_grad()
    result = evaluate(model, test_loader)
    model.epoch_end(epoch, result)
    history.append(result)
  return history

In [29]:
loss_fn = F.cross_entropy
learning_rate = 0.001

In [32]:
evaluate(model, test_loader)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:23: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


{'test_acc': 0.0985555574297905, 'test_loss': 2.3028807640075684}

In [33]:
history = fit(20, learning_rate, model, train_loader, test_loader)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:23: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


Epoch [0], test_loss: 2.3028, test_acc: 0.1003
Epoch [1], test_loss: 2.3027, test_acc: 0.0992
Epoch [2], test_loss: 2.3027, test_acc: 0.0966
Epoch [3], test_loss: 2.3024, test_acc: 0.1013
Epoch [4], test_loss: 2.3025, test_acc: 0.0986
Epoch [5], test_loss: 2.3025, test_acc: 0.1015
Epoch [6], test_loss: 2.3023, test_acc: 0.1717
Epoch [7], test_loss: 2.3021, test_acc: 0.0966
Epoch [8], test_loss: 2.3011, test_acc: 0.1004
Epoch [9], test_loss: 2.2978, test_acc: 0.1221
Epoch [10], test_loss: 2.1431, test_acc: 0.2065
Epoch [11], test_loss: 1.6384, test_acc: 0.3308
Epoch [12], test_loss: 1.4859, test_acc: 0.3508
Epoch [13], test_loss: 1.4316, test_acc: 0.3518
Epoch [14], test_loss: 1.3979, test_acc: 0.3437
Epoch [15], test_loss: 1.3542, test_acc: 0.4661
Epoch [16], test_loss: 1.3053, test_acc: 0.5098
Epoch [17], test_loss: 1.2553, test_acc: 0.5546
Epoch [18], test_loss: 1.2342, test_acc: 0.5138
Epoch [19], test_loss: 1.1846, test_acc: 0.5837


In [36]:
history = fit(50, learning_rate, model, train_loader, test_loader)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:23: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


Epoch [0], test_loss: 1.0300, test_acc: 0.6747
Epoch [1], test_loss: 1.0134, test_acc: 0.6883
Epoch [2], test_loss: 0.9919, test_acc: 0.6734
Epoch [3], test_loss: 0.9722, test_acc: 0.6786
Epoch [4], test_loss: 0.9582, test_acc: 0.6836
Epoch [5], test_loss: 0.9340, test_acc: 0.6994
Epoch [6], test_loss: 0.9149, test_acc: 0.6912
Epoch [7], test_loss: 0.9092, test_acc: 0.6896
Epoch [8], test_loss: 0.8844, test_acc: 0.7086
Epoch [9], test_loss: 0.8748, test_acc: 0.7037
Epoch [10], test_loss: 0.8632, test_acc: 0.7315
Epoch [11], test_loss: 0.8371, test_acc: 0.7472
Epoch [12], test_loss: 0.8276, test_acc: 0.7418
Epoch [13], test_loss: 0.8372, test_acc: 0.7248
Epoch [14], test_loss: 0.7932, test_acc: 0.7584
Epoch [15], test_loss: 0.7886, test_acc: 0.7583
Epoch [16], test_loss: 0.7675, test_acc: 0.7604
Epoch [17], test_loss: 0.7623, test_acc: 0.7612
Epoch [18], test_loss: 0.7457, test_acc: 0.7600
Epoch [19], test_loss: 0.7392, test_acc: 0.7648
Epoch [20], test_loss: 0.7423, test_acc: 0.7575
Ep